In [1]:
library(brms) #predictability: Double hierearchical model
library(MCMCglmm) #HPDinterval
library(dplyr) #%>%
library(lmerTest) #lmer
#library(MuMIn) #r2
#library(merTools) #sim
library(sjPlot) #plot coeff
library(ggplot2) #ylim in plot
library(rstan) #model diagnostic

Warning message:
"le package 'brms' a été compilé avec la version R 4.1.2"
Le chargement a nécessité le package : Rcpp

Warning message:
"le package 'Rcpp' a été compilé avec la version R 4.1.2"
Loading 'brms' package (version 2.16.3). Useful instructions
can be found by typing help('brms'). A more detailed introduction
to the package is available through vignette('brms_overview').


Attachement du package : 'brms'


L'objet suivant est masqué depuis 'package:stats':

    ar


Warning message:
"le package 'MCMCglmm' a été compilé avec la version R 4.1.2"
Le chargement a nécessité le package : Matrix

Warning message:
"le package 'Matrix' a été compilé avec la version R 4.1.2"
Le chargement a nécessité le package : coda

Warning message:
"le package 'coda' a été compilé avec la version R 4.1.2"
Le chargement a nécessité le package : ape

Warning message:
"le package 'ape' a été compilé avec la version R 4.1.2"

Attachement du package : 'MCMCglmm'


L'objet suivant est masqué depuis 'pac

In [2]:
library(ggplot2)

In [3]:
library(parallel)
my.cores = detectCores()-1 #minus 1 to avoid bocking the computer
my.cores

[1] 11

In [4]:
path_ = 'G:/VPHI/Welfare/2- Research Projects/OFHE2.OriginsE2/DataOutput/TrackingSystem/ALLDATA_'
#df = read.csv(file.path(path_,'df_daily_aggregatedHA.csv'), header = TRUE, sep = ",")
df = read.csv(file.path(path_,'OFH_df_MVT_simpler3.csv'), header = TRUE, sep = ",")

df$HenID = as.factor(df$HenID)   
df$PenID = as.factor(df$PenID)
df$TrackingSystemID = as.factor(df$TrackingSystemID)
df$level = as.factor(df$level) 
df$CLASS = as.factor(df$CLASS) 
df$Treatment = as.factor(df$Treatment)
df$time = scale(df$WIB, center=FALSE, scale=sd(df$WIB, na.rm = TRUE)) #as not centering, we need to add this
df$time2_ = poly(df$time, degree=2,raw=TRUE)[,2]
df$time2 = scale(df$time2_, center=FALSE, scale=sd(df$time2_, na.rm = TRUE))

#scale: mean-centering of the environmental variable so that intercepts reflect average values for the HenID and ind. 
#df$temperature_C_avg_scale = scale(df$temperature_C_avg, center=TRUE, scale=TRUE)

#setting reference group
contrasts(df$Treatment)
df <- df %>% mutate(Treatment = relevel(Treatment, ref = "TRAN"))
contrasts(df$Treatment)

contrasts(df$CLASS)
df <- df %>% mutate(CLASS = relevel(CLASS, ref = "LEXP"))
contrasts(df$CLASS)
print(dim(df))
summary(df)

,TRAN
OFH,0
TRAN,1


,OFH
TRAN,0
OFH,1


,LEXP,LEXPLOST,MEXP,MEXPLOST,NewAfterEpi
EPI,0,0,0,0,0
LEXP,1,0,0,0,0
LEXPLOST,0,1,0,0,0
MEXP,0,0,1,0,0
MEXPLOST,0,0,0,1,0
NewAfterEpi,0,0,0,0,1


,EPI,LEXPLOST,MEXP,MEXPLOST,NewAfterEpi
LEXP,0,0,0,0,0
EPI,1,0,0,0,0
LEXPLOST,0,1,0,0,0
MEXP,0,0,1,0,0
MEXPLOST,0,0,0,1,0
NewAfterEpi,0,0,0,0,1


[1] 2123   91


     HenID             level      verification_daily_total_duration
 hen_112:  39   2021-04-18:  63   Min.   :32400                    
 hen_57 :  39   2021-05-02:  61   1st Qu.:54000                    
 hen_59 :  39   2021-05-09:  61   Median :54000                    
 hen_60 :  39   2021-05-16:  61   Mean   :52511                    
 hen_61 :  39   2021-05-23:  60   3rd Qu.:54000                    
 hen_78 :  39   2021-05-30:  60   Max.   :54000                    
 (Other):1889   (Other)   :1757                                    
 verification_daily_total_nbr_hour  dur_values        perc_duration_3_Zone
 Min.   : 9.00                     Length:2123        Min.   : 0.00       
 1st Qu.:15.00                     Class :character   1st Qu.:12.00       
 Median :15.00                     Mode  :character   Median :21.00       
 Mean   :14.59                                        Mean   :21.07       
 3rd Qu.:15.00                                        3rd Qu.:29.00       
 Max. 

In [5]:
sd(df$time2)
sd(df$time2_)
sd(df$time)

[1] 1

[1] 3.813324

[1] 1

In [6]:
#df$Matrix_proportions = data.matrix(df[,c('propindoor_duration_2_Zone','propindoor_duration_3_Zone', 'propindoor_duration_4_Zone', 'propindoor_duration_5_Zone')])

In [ ]:
iter_ = 10000
iter_wu = 8000
bind <- function(...) cbind(...) #cbind is reserve for multivariate in brms!
#put 4 as reference!
frml = bf(bind(propindoor_duration_4_Zone, propindoor_duration_2_Zone, propindoor_duration_3_Zone,
               propindoor_duration_5_Zone) ~ time + time2 + Treatment + time:time2:Treatment + (1|PenID/HenID))
fit = brm(frml, data = df, family = dirichlet(link = 'logit', link_phi = 'log'), 
         iter = iter_, warmup = iter_wu, thin = 30, chains = 3, cores = my.cores, seed = 12345)
summary(fit)
save(fit, file = file.path(path_, 'OFH_dirichlet_80birdsPenIDtimesquare.rda'))
#load(file.path(path_,"OFH_dirichlet_TEST.rda")) 

Compiling Stan program...

Start sampling



In [ ]:
#as hens are in a unique pen:
li_dfpred = list()
li_dfnewdata = list()
for (penid in unique(df$PenID)){
    print(penid)
    df_newdata = expand.grid(Treatment=c("OFH",'TRAN'),
                             PenID = penid,
                             HenID =  unique(df[df$PenID==penid,]$HenID), #unique(df$HenID), #sample(unique(df$HenID), 10),
                             time = unique(df$time),
                             time2 = unique(df$time2),
                             Treatment = unique(df$Treatment))
    #dim(df_newdata)
    #head(df_newdata, 2)
    #predict.MCMCglmm() : https://www.rdocumentation.org/packages/MCMCglmm/versions/2.32/topics/predict.MCMCglmm
    #credible intervals: https://stat.ethz.ch/pipermail/r-sig-mixed-models/2018q1/026408.html 
    df_pred = predict(fit, newdata=df_newdata, interval='confidence') #marginal=fit$Random$formula OR NULL?
    #dim(df_pred)
    #head(df_pred,3)
    li_dfpred = append(li_dfpred,list(df_pred))
    li_dfnewdata = append(li_dfnewdata,list(df_newdata))
}
df_pred_all = bind_rows(li_dfpred)
df_newdata_all = bind_rows(li_dfnewdata)
dim(df_pred_all)
dim(df_newdata_all)
write.csv(df_newdata, file.path(path_,'OFH_df_pred_all.csv'))
write.csv(df_newdata_all, file.path(path_,'OFH_df_newdata_all.csv'))

In [ ]:
#TODO NEXT ROUND: all birds, account for CLASS; health, weight,

In [ ]:
#dirichlet diagnotic pp_check
#https://discourse.mc-stan.org/t/using-pp-check-for-dirichlet-model/8033/6
#good to know: http://paul-buerkner.github.io/brms/reference/conditional_effects.html
#to read: https://www.google.com/imgres?imgurl=https%3A%2F%2Fars.els-cdn.com%2Fcontent%2Fimage%2F1-s2.0-S0169207021000583-gr1.jpg&imgrefurl=https%3A%2F%2Fwww.sciencedirect.com%2Fscience%2Farticle%2Fpii%2FS0169207021000583&tbnid=ot_CAfIcu_8oPM&vet=12ahUKEwiTl-ez1tX4AhWyPOwKHSVVAhMQMygkegUIARDvAQ..i&docid=medWEoPnTXoAaM&w=531&h=525&q=plot%20Dirichlet%20prediction%20with%20brms%20odel&ved=2ahUKEwiTl-ez1tX4AhWyPOwKHSVVAhMQMygkegUIARDvAQ